<a href="https://colab.research.google.com/github/vikas-kr-yadav/number_plate_detection/blob/main/Number_plate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import cv2
import numpy as np
import os
import random
import tensorflow as tf

In [16]:
pip install --upgrade --no-cache-dir gdown


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [17]:
!pip install path

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
!pip install tensorflow==2.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.7/489.7 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 KB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffers-23.3.3:
      Successfully uninstalled flatbuffers-23.3.3
  Attempting uninstall: tensorflow
    Foun

In [19]:
! gdown --id 10ZSIyr7ZdaGGKIFS0HxxlOJbw67YEW_o

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=10ZSIyr7ZdaGGKIFS0HxxlOJbw67YEW_o
To: /content/archive.zip
100% 213M/213M [00:06<00:00, 34.0MB/s]


In [20]:
import zipfile
import os
import shutil
with zipfile.ZipFile("archive.zip","r") as f:
  f.extractall('.')

In [25]:
# Step 1: Prepare the data
# Load the dataset
data_dir = '/content/images'
images = []
labels = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.jpg'):
            img_path = os.path.join(root, file)
            img = cv2.imread(img_path)
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, img_thresh = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            img_resized = cv2.resize(img_thresh, (128, 64))
            images.append(img_resized)
            labels.append(file.split('.')[0])

In [26]:
# Split the dataset into training and testing sets
data = list(zip(images, labels))
random.shuffle(data)
train_data = data[:int(len(data)*0.8)]
test_data = data[int(len(data)*0.8):]

train_images = np.array([item[0] for item in train_data])
train_labels = [item[1] for item in train_data]
test_images = np.array([item[0] for item in test_data])
test_labels = [item[1] for item in test_data]
test_images, test_labels = images[num_train:], labels[num_train:]

In [28]:
# Step 2: Preprocess the data
# Convert the images to float and normalize them
train_images = np.array(train_images).astype('float32') / 255
test_images = np.array(test_images).astype('float32') / 255

# Reshape the images to match the input shape of the model
train_images = np.reshape(train_images, (train_images.shape[0], 64, 128, 1))
test_images = np.reshape(test_images, (test_images.shape[0], 64, 128, 1))


In [29]:
# Step 3: Build the model
# Define the model architecture
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 128, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(set(labels))+1, activation='softmax'))


In [30]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert the labels to one-hot encoding
char_set = set(''.join(labels))
char_to_int = dict((c, i) for i, c in enumerate(char_set))
int_to_char = dict((i, c) for i, c in enumerate(char_set))
train_labels_encoded = np.array([np.eye(len(char_set)+1)[[char_to_int[c] for c in label]] for label in train_labels])
test_labels_encoded = np.array([np.eye(len(char_set)+1)[[char_to_int[c] for c in label]] for label in test_labels])


In [41]:
# Train the model
model.fit(train_images, train_labels_encoded, epochs=10, batch_size=32, validation_data=(test_images, test_labels_encoded))


Epoch 1/10


ValueError: ignored

In [42]:
# Step 4: Evaluate the model
# Evaluate the model on the testing set
loss, accuracy = model.evaluate(test_images, test_labels_encoded)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

/usr/local/lib/python3.9/dist-packages/keras/utils/generic_utils.py:239: RuntimeWarning: divide by zero encountered in log10
  """


OverflowError: ignored

In [44]:
img_test = cv2.imread('/content/images/Cars100.png')
img_test_gray = cv2.cvtColor(img_test, cv2.COLOR_BGR2GRAY)
_, img_test_thresh = cv2.threshold(img_test_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
img_test_resized = cv2.resize(img_test_thresh, (128, 64))

In [46]:
# Preprocess the image for prediction
img_test_resized = img_test_resized.astype('float32') / 255
img_test_resized = np.reshape(img_test_resized, (1, 64, 128, 1))

In [47]:
# Use the model for prediction
prediction = model.predict(img_test_resized)
prediction = np.argmax(prediction, axis=-1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 179ms/step


In [59]:
# Convert the prediction to characters
predicted_label = ''
for p in prediction[0]:
  if p == len(char_set):
    break
predicted_label += int_to_char[p]

TypeError: ignored

In [54]:
# Print the predicted label
print('Predicted Label:', predicted_label)

Predicted Label: 
